In [ ]:
from bs4 import BeautifulSoup
import requests
from csv import writer
import time
import random
from lxml import etree as et
import urllib.request
import re
import numpy as np

In [ ]:
pages_url = []
url = "https://batdongsan.vn/ban-nha/p"
for i in range(495):
    page_url = url + str(i+1)
    pages_url.append(page_url)

In [ ]:
def download_html(url):
    with urllib.request.urlopen(url) as response:
        html = response.read()
        html = html.decode('utf-8')
    response.close()
    return html

In [ ]:
def get_house_url(soup_page_url):           
    source = download_html(soup_page_url)            
    soup_page_url = BeautifulSoup(source, 'html.parser')
    house_url_pages = soup_page_url.find('div',{'class':'datalist'}).findAll('div', {'class':'name'})
    for i in range(len(house_url_pages)):
        link = house_url_pages[i].find('a')['href']
        house_url_pages[i]=link
    # Trả về link của các căn nhà trong 1 trang.
    return house_url_pages                  

In [ ]:
def scrap (location, user, price, infor, title, time, link_each_house_pages):  
    
    data={}
    # Lấy link căn nhà.
    data['Link_house'] = link_each_house_pages

    # Lấy tên người đăng.
    try:
        data['user'] = user.get_text()
    except:
        data['user'] = None

    # Lấy link người đăng.
    try: 
        data['user_link'] = user.find('a')['href']
    except:
        data['user_link'] = None
    
    # Lấy thời gian của bài viết.
    data['time'] = time

    # Lấy thông tin Tỉnh và Huyện.
    try: 
        data['Tinh/TP'] = location[2].get_text()
        data['Quan/Huyen'] = location[3].get_text()
    except:
        data['Tinh/TP'] = None
        data['Quan/Huyen'] = None
    for i in infor:
        # Lấy các thông tin chi tiết trong class body có diện tích, phòng ngủ, phòng vệ sinh, hướng nhà, hướng ban công.
        data[i.find('strong').text] = None
        data[i.find('strong').text] = str(re.search(r'[\d.]+',i.find(text=True, recursive=False).strip()))
        if (i.find('strong').text != 'Diện tích' or i.find('strong').text != 'Phòng ngủ' or i.find('strong').text != 'Phòng WC'):
            data[i.find('strong').text] = i.find(text=True, recursive=False).strip()
    
    # Lấy thông tin về giá nhà trong class price.
    data['price'] = None
    price = price.contents[0].strip()
    if price != 'Thỏa thuận':
        price_ = re.match(r'^([\d.,]+)\s*(\D+)$',price)
        unit = price_.group(2) 
        if unit == 'triệu':
            data['price'] =  float(price_.group(1)) / 1000
        else:
            if unit == 'tỷ':
                data['price'] =  price_.group(1)
    else:
        matches = re.findall(r'(\d+([\.,]\d+)?)\s*(tỷ|tỉ|ty)(\d{0,3})?', title, re.IGNORECASE)
        if matches:
            prices = [match[0] for match in matches]
            data['price'] = prices[-1]
    
    # Lấy thông tin giá nhà trong title.
    billion = 0  
    million = 0  
    matches = re.findall(r'(\d+\w*([.,]{1}\d*\w*){0,1}){1}\s*(tỷ|tỉ|ty){1}\s*(\d*\w*)', title, re.IGNORECASE)
    if matches:
        try:
            billion = str(matches[0][0]).replace(',', '.')
        except:
            a = 0
        try:
            if matches[0][-1] != '':
                million = str(matches[0][-1]).replace(',', '.')
        except:
            a = 0
        try:
            data['price_title'] = str(float(billion) + int(million)/ 1000)  
        except:
            if matches != '':
                if million == 0:
                    data['price_title'] = str(billion)
                else:
                    data['price_title']  = str(billion) + '.' + str(million)
    else:
        matches = re.findall(r'(\d+)\s*(triệu|trieu|tr){1}', title, re.IGNORECASE)
        if matches:
            data['price_title'] = str(float(matches[0][0]) / 1000)

    # Lấy số tầng của căn nhà từ title.
    data['Tầng'] = None
    matches = re.findall(r'(\d+)\s*(tầng)', title, re.IGNORECASE)
    if matches:
        floors = [match[0] for match in matches]
        data['Tầng'] = floors[0]
    else:
        matches = re.findall(r'(\d+)\s*(lầu)', title, re.IGNORECASE)
        if matches:
            floors = [match[0] for match in matches]
            data['Tầng'] = int(floors[0]) + 1
    
    # Trả về một từ điển với các thông tin của căn nhà.
    return data


In [ ]:
def extract_house_info(text):       # Lấy thông tin căn nhà từ content.
    info = {}

    # Lấy thông tin giá.
    info['giá'] = 0
    billion = 0  
    million = 0  
    matches = re.findall(r'(\d+\w*([.,]{1}\d*\w*){0,1}){1}\s*(tỷ|tỉ|ty){1}\s*(\d*\w*)', text, re.IGNORECASE)
    if matches:
        try:
            billion = str(matches[0][0]).replace(',', '.')
        except:
            a = 0
        try:
            if matches[0][-1] != '':
                million = str(matches[0][-1]).replace(',', '.')
        except:
            a = 0
        try:
            info['giá'] = str(float(billion) + int(million)/ 1000)  
        except:
            if matches != '':
                if million == 0:
                    info['giá'] = str(billion)
                else:
                    info['giá']  = str(billion) + '.' + str(million)
    else:
        matches = re.findall(r'(\d+)\s*(triệu|trieu|tr){1}', text, re.IGNORECASE)
        if matches:
            info['giá'] = str(float(matches[0][0]) / 1000)

    # Lấy thông tin về diện tích.
    match = re.search(r'((DT:|diện tích:)\s*[\d.]+)|([\d.]\s*+(m2)+)', text, re.IGNORECASE)
    if match:
        info['Diện tích'] = match.group(1)

    # Lấy thông tin về số phòng ngủ.
    match = re.findall(r'(\d+){1}\s*(pn|phòng ngủ|phong ngu){1}', text, re.IGNORECASE)
    if match:
        PN = 0
        for i in range(len(match)):
            PN += int(match[i][0])
        info['Phòng ngủ'] = PN    

    # Lấy thông tin về số phòng WC.
    match = re.findall(r'(\d+){1}\s*(wc|phong wc|phong ve sinh|phòng vệ sinh){1}', text, re.IGNORECASE)
    if match:
        WC = 0
        for i in range(len(match)):
            PN += int(match[i][0])
        info['Phòng WC'] = WC

    # Lấy thông tin về số tầng.
    match = re.search(r'(\d+)\s*(tầng)', text, re.IGNORECASE)
    if match:
        info['Số tầng'] = match.group(1)
    else:
        info['Số tầng'] = 0
    match = re.search(r'(\d+)\s*(lầu)', text, re.IGNORECASE)
    if match:
        info['Số lầu'] = match.group(1)
    else:
        info['Số lầu'] = 0
    match = re.search(r'(\d+)\s*(trệt)', text, re.IGNORECASE)
    if match:
        info['trệt'] = match.group(1)
    else:
        info['trệt'] = 0
    
    # Chuyển đổi câc số lầu và trệt thành tầng.
    if info['Số tầng'] == 0:
        if info['Số lầu'] == 0 and info['trệt'] != 0 :
            info['Số tầng'] = int(info['trệt']) + 1
        if info['trệt'] == 0 and info['Số lầu'] != 0:
            info['Số tầng'] = int(info['Số lầu']) + 1
        if info['trệt'] != 0 and info['Số lầu'] != 0:
            info['Số tầng'] = int(info['Số lầu']) + int(info['trệt'])

    # Kiểm tra sự có mặt của sân
    match = re.search(r'([\w]+\s*(sân))|((sân)\s[\w]+)', text, re.IGNORECASE)
    if match:
        info['Có sân'] = match.group()

    # Kiểm tra có gần bệnh viện không
    match = re.search(r'([\w\s]+\s*(bệnh viện))|((bệnh viện)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần bệnh viện'] = match.group()

    # Kiểm tra có gần chợ không.
    match = re.search(r'([\w\s]+\s*(chợ|siêu thị))|((chợ|siêu thị)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần chợ'] = match.group()

    # Kiểm tra có gần trường học không.
    match = re.search(r'([\w\s]+\s*(trường))|((trường)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Gần trường'] = match.group()

    # kiểm tra có thuộc khu dân trí không.
    match = re.search(r'((dân trí)\s[\w\s]+)|([\w\s]+\s*(dân trí))', text, re.IGNORECASE)
    if match:
        info['Dân trí'] = match.group()

    # kiểm tra khu vục có an ninh không.
    match = re.search(r'((an ninh)\s[\w\s]+)|([\w\s]+\s*(an ninh))', text, re.IGNORECASE)
    if match:
        info['An ninh'] = match.group()

    # Kiểm tra nhà có thuận tiện buôn bán không.
    match = re.search(r'([\w\s]+\s*(kinh doanh|buôn bán))|((kinh doanh|buôn bán)\s[\w\s]+)', text, re.IGNORECASE)
    if match:
        info['Kinh doanh'] = match.group()
    # Các lệnh kiểm tra trên kết quả trả về là một chuổi thông tin.

    # Lấy địa chỉ của căn nhà.
    match = re.search(r'((vị trí:|địa điểm:|khu vục:số|đường|quận|huyện|tỉnh|thành phố|tp)[\s\w,]+)', text, re.IGNORECASE)
    if match:
        info['Địa điểm'] = match.group()
    
    return info

In [ ]:
def merge_dicts(library1, library2):
    merge_dicts = {}

    # Gộp dict 1.
    for key, value in library1.items():
        merge_dicts[key] = value

    # Gộp dict 2.
    for key, value in library2.items():
        merge_dicts[key] = value

    # trả về hai dict đã gộp.
    return merge_dicts

In [ ]:
def scrape_this(link_each_house_pages):            
    link = link_each_house_pages
    # Download link căn nhà.
    source_house = download_html(link) 
    soup_house = BeautifulSoup(source_house,'html.parser')

    # Thông tin của title và class body sẽ được luu vào đây.  
    data1={}      

    # Thông tin của content sẽ được lưu vào đây.
    data2={}

    # Nơi chưa thông tin sao khi kết hợp lại.
    data=[]

    # Lấy các tham số cần thiết để chuẩn bị lấy thông tin.
    try:
        location = soup_house.find('ul',{'class':'uk-breadcrumb'}).findAll('li')
    except:
        location=''
    try:
        user = soup_house.find('div', {'class':'name'})
    except:
        user=''
    try:
        price = soup_house.find('strong', class_='price')
    except:
        price=''
    try:
        info = soup_house.find('div', {'class':'body'}).findAll('ul',{'class':'uk-list'})
    except:
        info=''
    try:
        time = info[1].findAll('li')
        info = info[0].findAll('li')
    except:
        time = info[0]
    try:
        title = soup_house.find('h1',{'class':'uk-panel-title'}).get_text()
    except:
        title=''
    try:
        content = soup_house.find('div',{'class':'content'}).get_text()
    except:
        content=''
  
    data1 = scrap(location, user, price, info, title, time, link)
    data2 = extract_house_info(content)
    
    # Gộp hai từ điển lại với nhau và thêm vào data.
    data.append(merge_libraries(data1, data2))
    return data

In [ ]:
# chứa dữ liệu cuối cùng
data=[]

# Thực hiện lấy thông tin từng trang một.
for i in range(len(pages_url)): 
    print('Đang lấy thông tin trang thứ ',i)                     
    link_house = get_house_url(pages_url[i]) 

    # Thực hiện lấy thông tin từng nhà một.
    for j in link_house: 
        try:     
            data.extend(scrape_this(j))       
        except:
            continue


In [ ]:
import pandas as pd

# Lưu dataframe vào file csv
data_=pd.DataFrame(data)
data_.to_csv("D:\\NM_KDL\\practice\\Project final\\data\\file_full.csv", index=False)

In [ ]:
data.head()

,Link_house,user,user_link,time,Tinh/TP,Quan/Huyen,Diện tích:,Địa chỉ:,price,price_title,...,Phòng ngủ,Có sân,Địa điểm,Hướng nhà:,Kinh doanh,Hướng ban công:,An ninh,Phòng WC,Dân trí,Gần bệnh viện
0,https://batdongsan.vn/can-ban-nha-kien-hung-ha...,Dương Nguyễn,https://batdongsan.vn/duongnguyen.181874,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Hà Đông,35 m,"Quận Hà Đông, Hà Nội",3.2,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://batdongsan.vn/nha-moi-sai-dong-long-bi...,Bùi Mạnh Cương,https://batdongsan.vn/buimanhcuong.189440,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Long Biên,NaN,Long Biên,5.2,5.NHỈNH,...,4.0,có sân,"tỉnh như đường 5B, đường Nguyễn Văn Linh",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://batdongsan.vn/ban-gap-nha-pho-truong-c...,lưu văn anh,https://batdongsan.vn/luuvananh.192740,"[<li><i class=""uk-icon-angle-right""></i><span>...",Hà Nội,Đống Đa,34 m,Hà Nội,3,3.0,...,4.0,sân bóng,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://batdongsan.vn/nha-1tret-2lau-hem-xe-ho...,Hồ Ngọc Long,https://batdongsan.vn/hongoclong.52923,"[<li><i class=""uk-icon-angle-right""></i><span>...",TP Hồ Chí Minh,Thủ Đức,50 m,"Đường Số 2, Phường Trường Thọ, Quận Thủ Đức, T...",5.6,5.6,...,3.0,sân phơi,"đường số 2 Trường Thọ Thủ Đức, giá 5,6 tỷ\r\nV...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,https://batdongsan.vn/ban-nha-hxh-sat-duong-so...,Dương Trí Dũng,https://batdongsan.vn/duongtridung.178383,"[<li><i class=""uk-icon-angle-right""></i><span>...",TP Hồ Chí Minh,Gò Vấp,64 m,Đường Số 8,8.8,8.8,...,5.0,sân thượng,Vị trí: nhà HXH sát Đường Số 8 Phường 11 Gò Vấ...,Nam,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9744 entries, 0 to 870
Data columns (total 30 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Link_house       9744 non-null   object 
 1   user             9744 non-null   object 
 2   user_link        9744 non-null   object 
 3   time             9744 non-null   object 
 4   Tinh/TP          9744 non-null   object 
 5   Quan/Huyen       9744 non-null   object 
 6   Diện tích:       8954 non-null   object 
 7   Địa chỉ:         5055 non-null   object 
 8   price            9374 non-null   object 
 9   price_title      6982 non-null   object 
 10  Tầng             3796 non-null   float64
 11  giá              9744 non-null   object 
 12  Diện tích        1790 non-null   object 
 13  Số tầng          9744 non-null   int64  
 14  Số lầu           9744 non-null   int64  
 15  trệt             9744 non-null   int64  
 16  Gần chợ          3786 non-null   object 
 17  Gần trường     

In [ ]:
# Lấy thời gian cho cột time
data['time'] = data['time'].str.extract(r'datetime="([^"]+)"')

# Tạo thuộc tính mới là PN từ hai cột phòng ngủ lấy từ hai nguồn dữ liệu khác nhau. Ưu tiên lấy của class body trước sau đó tới của content.
data['PN'] = data.apply(lambda row: str(row['Phòng ngủ:']) if pd.notnull(row['Phòng ngủ:']) else str(row['Phòng ngủ']), axis=1)

# Thuộc tính WC, area, floor cũng tương tự.
data['WC'] = data.apply(lambda row: str(row['Phòng WC:']) if pd.notnull(row['Phòng WC:']) else str(row['Phòng WC']), axis=1)
data['area'] = data.apply(lambda row: row['Diện tích:'] if pd.notnull(row['Diện tích:']) else row['Diện tích'], axis=1)
data['floor'] = data.apply(lambda row: str(row['Tầng']) if pd.notnull(row['Tầng']) else str(row["Số tầng"]), axis=1)

# Chuyển đổi các chuổi kiểm tra thành giá trị True, Flase.
data['Gần trường'] = data['Gần trường'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Kinh doanh'] = data['Kinh doanh'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Gần chợ'] = data['Gần chợ'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Gần bệnh viện'] = data['Gần bệnh viện'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['An ninh'] = data['An ninh'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Có sân'] = data['Có sân'].str.contains('[a-zA-Z]', regex=True).fillna(False)
data['Dân trí'] = data['Dân trí'].str.contains('[a-zA-Z]', regex=True).fillna(False)

In [ ]:
# Gộp các thông tin kiểm tra này lại thành một cột tien_ich và có giá trị bằng tổng các true.
data_temp=data[['Có sân','Dân trí','An ninh','Kinh doanh','Gần chợ','Gần trường','Gần bệnh viện']]
data['Tien_ich'] = data_temp.apply(lambda row: sum(row), axis=1)

# Drop các dòng không cần thiết
data.drop('Có sân', inplace=True, axis=1)
data.drop('Dân trí', inplace=True, axis=1)
data.drop('An ninh', inplace=True, axis=1)
data.drop('Kinh doanh', inplace=True, axis=1)
data.drop('Gần chợ', inplace=True, axis=1)
data.drop('Gần trường', inplace=True, axis=1)
data.drop('Gần bệnh viện', inplace=True, axis=1)

In [ ]:
# Do cột PN, WC, floor mang kiểm object nên cần chuyển nan thành NaN nếu có. Để đảm bảo.
data['PN'] = data['PN'].replace('nan', np.nan)
data['WC'] = data['WC'].replace('nan', np.nan)
data['floor'] = data['floor'].replace('nan', np.nan)
data['area'] = data['area'].replace('nan', np.nan)


In [ ]:
# Drop các thuộc tính bị dư thừa hoặc không cần thiết cho quá trình phân tích dữ liệu sau này.
data.drop('Link_house', inplace= True, axis=1)
data.drop('user', inplace=True, axis=1)
data.drop('user_link', inplace=True, axis=1)
data.drop('time', inplace=True, axis=1)
data.drop('Tầng', inplace=True, axis=1)
data.drop('Số tầng', inplace=True, axis =1)
data.drop('Số lầu', inplace=True, axis =1)
data.drop('trệt', inplace=True, axis =1)
data.drop('Diện tích:', inplace=True, axis=1)
data.drop('Diện tích', inplace=True, axis=1)
data.drop('Phòng ngủ:', inplace=True, axis=1)
data.drop('Phòng WC:', inplace=True, axis=1)
data.drop('Phòng ngủ', inplace=True, axis=1)
data.drop('Phòng WC', inplace=True, axis=1)
data.drop('Địa điểm', inplace=True, axis=1)
data.drop('Địa chỉ:', inplace=True, axis=1)
data.drop('Hướng nhà:', inplace=True, axis=1)
data.drop('Hướng ban công:', inplace=True, axis=1)

In [ ]:
# Do PN, WC, area trong đó có chữ nên bây giờ chúng ta chỉ lấy các số mà thôi.
data['PN'] = data['PN'].str.replace(' PN', '')
data['WC'] = data['WC'].str.replace(' WC','')
data['area'] = data['area'].str.replace(' m','')
data['area'] = data['area'].fillna('')
data['area'] = data['area'].apply(lambda x: ("".join(re.findall(r'[\d.,]*', x))))
data['area']=data['area'].replace('',np.nan)

In [ ]:
# Thay đổi dấu "," thành dấu "." và chuyển kiểu dữ liệu về float
data['area']=data['area'].replace(',', '.', regex=True)
data['area']=data['area'].astype(float)
data['PN']=data['PN'].astype(float)
data['WC']=data['WC'].astype(float)
data['floor']=data['floor'].astype(float)

# Các nhà có floor = 0 thì ta chuyển lại thành NaN sau đó chuyển chúng thành 1.
data['floor'] = data['floor'].replace(0.0,np.nan)
data['floor'] = data['floor'].replace(np.nan, 1)

# Kiểm tra lại tông tin của các thuộc tính
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9744 entries, 0 to 870
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Tinh/TP      9744 non-null   object 
 1   Quan/Huyen   9744 non-null   object 
 2   price        9374 non-null   object 
 3   price_title  6982 non-null   object 
 4   giá          9744 non-null   object 
 5   PN           7660 non-null   float64
 6   WC           6325 non-null   float64
 7   area         9129 non-null   float64
 8   floor        9744 non-null   float64
 9   Tien_ich     9744 non-null   int64  
dtypes: float64(4), int64(1), object(5)
memory usage: 837.4+ KB


In [ ]:
# Bây giờ tới việc xử lý giá nhà.
pattern = r'(\d+(?:\.\d+)?)'

# Áp dụng regex vào cột "price_title" và cột "giá"(là giá lấy từ content) để chỉ lấy các giá trị bằng số
data['price_title'] = data['price_title'].apply(lambda x: re.findall(pattern, str(x))[0] if re.findall(pattern, str(x)) else x)
data['giá'] = data['giá'].apply(lambda x: re.findall(pattern, str(x))[0] if re.findall(pattern, str(x)) else x)

In [ ]:
# Chuyển đổi các dấu "," thành "." và định dạng lại kiểu của có là chuổi.
data['giá']=data['giá'].apply(lambda x: float(str(x).replace(',','.')))
data['price']=data['price'].apply(lambda x: float(str(x).replace(',','.')))
data['price_title']=data['price_title'].apply(lambda x: float(str(x).replace(',','.')))

In [ ]:
# Tạo cột 'price_' và sao chép dữ liệu từ cột 'price'.
# Chẳng hạn như nếu số giá mang giá trị trong khoảng tỷ thì có thể người dùng đã nhập lộn đơn vị nên mình sẽ chia cho 1000000.
data.loc[(data['price'] >= 1000000) & (data['price'] <= 15000000), 'price'] /= 1000000
data['price_'] = data['price']

# Kiểm tra giá trị của cột 'price_' và thực hiện các điều kiện.
# Để phòng trường hợp người dùng nhập sai số lượng số không, hoặc sai vị trí dấu chấm.
for i, row in data.iterrows():
    if row['price_'] < 0.5:
        if max(row['giá'], row['price_title']) >=1:
            max_value = max(row['giá'], row['price_title'])
            data.at[i, 'price_'] = max_value
        else:
            data.at[i, 'price_'] = row['price']   
    elif 100 <= row['price_'] <= 1000:
        if 1 <= row['giá'] <= 100 and 1 <= row['price_title'] <= 100:
            data.at[i, 'price_'] = row['giá']
        else:
            if 1 <= row['giá'] <= 100:
                data.at[i, 'price_'] = row['giá']
            else: 
                if 1 <= row['price_title'] <= 100:
                    data.at[i, 'price_'] = row['price_title']
                else:
                    data.at[i, 'price_'] = row['price'] 
    elif 1000 <= row['price_'] <= 10000:
        if 1 <= row['giá'] <= 100 and 1 <= row['price_title'] <= 100:
            data.at[i, 'price_'] = row['giá']
        else:
            if 1 <= row['giá'] <= 100:
                data.at[i, 'price_'] = row['giá']
            else: 
                if 1 <= row['price_title'] <= 100:
                    data.at[i, 'price_'] = row['price_title']
                else:
                    data.at[i, 'price_'] = row['price']          
    elif row['price_'] > 10000:
        if 1 <= row['giá'] <= 500 and 1 <= row['price_title'] <= 500:
            data.at[i, 'price_'] = row['giá']
        elif 1 <= row['price_title'] <= 500:
            data.at[i, 'price_'] = row['price_title']
        elif 1 <= row['giá'] <= 500:
            data.at[i, 'price_'] = row['giá']
        else:
            data.at[i, 'price_'] = row['price'] 
    else:
        data.at[i, 'price_'] = row['price']     





In [ ]:
# Chuyển đổi các số 0.0 thành NaN và xóa các dòng có giá trị null.
data['price_'] = data['price_'].replace(0.0, np.nan)
data = data.dropna(subset=['price_'])

In [ ]:
# Xóa các cột giá không cần thiết nữa.
data.drop('price', inplace=True, axis=1)
data.drop('giá', inplace=True, axis=1)
data.drop('price_title', inplace=True, axis=1)

In [ ]:
# Nhận thấy không có căn nhà nào có diện tích thấp hơn 10m2.
# Các nhà có giá tiền thấp hơn 0.3 toàn bộ điều là nhà thuê, và cũng không có nhà lớn hơn 1000 tỷ.
data = data[data['price_'] >= 0.3]
data = data[data['area'] >= 10]
data = data[data['price_'] < 1000]

In [ ]:
data.head()

,Tinh/TP,Quan/Huyen,PN,WC,area,floor,Tien_ich,price_
0,Hà Nội,Hà Đông,NaN,NaN,35.0,5.0,2,8.00
2,Hà Nội,Đống Đa,4.0,NaN,34.0,4.0,2,5.45
3,TP Hồ Chí Minh,Thủ Đức,3.0,NaN,50.0,3.0,2,5.45
4,TP Hồ Chí Minh,Gò Vấp,5.0,NaN,64.0,5.0,2,7.80
5,Đồng Nai,Biên Hòa,2.0,NaN,200.0,1.0,2,18.00


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8707 entries, 0 to 870
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Tinh/TP     8707 non-null   object 
 1   Quan/Huyen  8707 non-null   object 
 2   PN          7035 non-null   float64
 3   WC          5862 non-null   float64
 4   area        8707 non-null   float64
 5   floor       8707 non-null   float64
 6   Tien_ich    8707 non-null   int64  
 7   price_      8707 non-null   float64
dtypes: float64(5), int64(1), object(2)
memory usage: 612.2+ KB


In [ ]:
data.isnull().sum()

Tinh/TP          0
Quan/Huyen       0
PN            1672
WC            2845
area             0
floor            0
Tien_ich         0
price_           0
dtype: int64

In [ ]:
data.to_csv("Data_Realestate.csv", index=False)